In [1]:
!nvidia-smi
import torch
import os
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics import YOLO
from mask_attack.utils.dataset import CustomDataset
# os.chdir(path='E://bmx/YoloCAM')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = dict(model="../models/yolov8-gtsrb.pt", data="./data.yaml")
trainer = DetectionTrainer(overrides=args)
trainer.setup_model()
trainer.set_model_attributes()
trainer.model.to(device=device)
trainer.model.train()
detector = YOLO(model="../models/yolov8-gtsrb.pt")
detector.to(device=device, dtype=torch.float32).eval()

Thu May 22 18:55:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  |   00000000:01:00.0  On |                  Off |
|  0%   48C    P8             16W /  450W |   10809MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [2]:
from analyzer.utils import analysis_all_layer_cross_classes
# key_filters = analysis_all_layer_cross_classes(trainer)
key_filters = {
    'backbone_c2f1.cv1': [10, 13, 16],
    'backbone_c2f1.cv2': [27],
    'backbone_c2f1.m.0.cv1': [],
    'backbone_c2f1.m.0.cv2': [],
    'backbone_c2f2.cv1': [7, 18, 21],
    'backbone_c2f2.cv2': [21, 38, 45],
    'backbone_c2f2.m.0.cv1': [6, 9, 12, 28, 29, 31],
    'backbone_c2f2.m.0.cv2': [6, 7, 24],
    'backbone_c2f2.m.1.cv1': [7],
    'backbone_c2f2.m.1.cv2': [],
    'backbone_c2f3.cv1': [12, 20, 21, 22, 39, 58, 61],
    'backbone_c2f3.cv2': [0, 41, 96],
    'backbone_c2f3.m.0.cv1': [28],
    'backbone_c2f3.m.0.cv2.conv': [51],
    'backbone_c2f3.m.1.cv1': [],
    'backbone_c2f3.m.1.cv2': [],
    'backbone_c2f4.cv1': [48],
    'backbone_c2f4.cv2': [91, 122],
    'backbone_c2f4.m.0.cv1': [73, 84],
    'backbone_c2f4.m.0.cv2': [120],
    'neck_c2f1.cv1': [25],
    'neck_c2f1.cv2': [],
    'neck_c2f1.m.0.cv1': [40, 59],
    'neck_c2f1.m.0.cv2': [],
    'neck_c2f2.cv1': [1, 52],
    'neck_c2f2.cv2': [],
    'neck_c2f2.m.0.cv1': [24],
    'neck_c2f2.m.0.cv2': [],
    'neck_c2f3.cv1': [7, 114],
    'neck_c2f3.cv2': [44],
    'neck_c2f3.m.0.cv1': [],
    'neck_c2f3.m.0.cv2': [47, 59],
    'neck_c2f4.cv1': [],
    'neck_c2f4.cv2': [228],
    'neck_c2f4.m.0.cv1': [],
    'neck_c2f4.m.0.cv2': [10, 126]
}

In [ ]:
from mask_attack.utils.attacker import Attacker
origin_dataset = CustomDataset(images_dir_path = "../datasets/gtsrb_origin/0/images", 
                              labels_dir_path = "../datasets/gtsrb_origin/0/labels",
                              image_width=640,
                              image_height=640)
attacker = Attacker(trainer=trainer, dataset=origin_dataset, batch_size=16)
sample = origin_dataset[0]
# print(sample['image'].size())
img_adv_tensor = attacker.single_filter_attack(sample=sample,
                              lr=0.001, epsilon=0.0001, num_iter=100,
                              lambda1=10, lambda2=1, lambda3=10,
                              key_filters=key_filters)

Step   0, LossT,1,2,3: -3846.9255371094 0.0000000000 3766.7797851562 8.0145683289
Step   1, LossT,1,2,3: -3899.7221679688 0.6071841121 3823.4377441406 8.2356300354
Step   2, LossT,1,2,3: -3949.1877441406 0.4840473235 3854.1770019531 9.9851417542
Step   3, LossT,1,2,3: -4012.4069824219 0.4233519435 3868.7070312500 14.7933397293
Step   4, LossT,1,2,3: -4043.1542968750 0.4432381988 3867.1052246094 18.0481491089
Step   5, LossT,1,2,3: -4055.0432128906 0.4655558467 3861.9252929688 19.7773418427
Step   6, LossT,1,2,3: -4069.7846679688 0.4778690934 3865.5163574219 20.9047069550
Step   7, LossT,1,2,3: -4084.4450683594 0.4837365746 3870.2561035156 21.9026336670
Step   8, LossT,1,2,3: -4092.1291503906 0.4877621830 3872.5334472656 22.4473457336
Step   9, LossT,1,2,3: -4101.8974609375 0.4917635322 3875.7963867188 23.1018810272
Step  10, LossT,1,2,3: -4110.0561523438 0.4950864911 3880.2121582031 23.4794921875
Step  11, LossT,1,2,3: -4114.2124023438 0.4986122251 3881.2436523438 23.7954750061
Step  1

In [8]:
from PIL import Image
import cv2
# print(img_adv_tensor)
img_adv_array = (img_adv_tensor.detach().squeeze(0) * 255).to(torch.uint8).permute(1, 2, 0).cpu().numpy()
img_adv = Image.fromarray(img_adv_array)
img_adv.save("./result/filter_attack/output_image.png")  # 保存为 PNG 格式